# UC-Berkeley Milling Dataset
> This notebook demonstrates how we can use PyPHM with the UC-Berkeley Milling Dataset. We also conduct some basic data exploration to help the reader understand the data.

The UC-Berkeley Milling Dataset is a collection of milling machine signals, collected during metal machining. The dataset contains 167 cuts on four different type of metal. Periodic measurement of tool wear was also recorded. Further details are found in the Data Exploration section, below.

The dataset is available from the [NASA Prognostic Data Repository](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#milling). Consider citing the creators of the dataset if you use it in academic research.

# Quickstart
Use PyPHM to quickly download and prepare the dataset.

Start by importing PyPHM, numpy, pandas, and pathlib.

In [1]:
from pyphm.datasets.milling import MillingPrepMethodA
import numpy as np
import pandas as pd
from pathlib import Path

We will use `MillingPrepMethodA` to prepare the dataset. This method is similar to that described in Cheng et al. [cheng2019multisensory] and von Hahn et al. [].

Instantiate the `MillingPrepMethodA` class. Dowload the dataset by setting `download` to `True`. Keep the sliding window length (`window_size`) to 64, and the amount each window moves (`stride`) to 64. These are also the default values, if you leave these variables blank.

In [3]:
# define the location of where the raw data folders will be kept.
# e.g. the milling data will be in path_data_raw_folder/milling/ 
path_data_raw_folder = Path(Path.cwd().parent / 'data/raw/' )
print(path_data_raw_folder)

# create the path_data_raw_folder if it does not exist
path_data_raw_folder.mkdir(parents=True, exist_ok=True)

# instantiate the MillingPrepMethodA class and download data if it does not exist
mill = MillingPrepMethodA(root=path_data_raw_folder, download=True, window_len=64, stride=64)

c:\_Python\PyPHM\data\raw


## Create Numpy Arrays
With the dataprep class instantiated, we can now prepare the dataset by calling methods from the class.

First, let's call the `create_xy_arrays` method. This will create two numpy arrays. The `x` array contains the windowed signals from the milling machine. The `y` array contains the labels and meta-data for each window.

In [4]:
x, y = mill.create_xy_arrays()
print("x.shape", x.shape)
print("y.shape", y.shape)

x.shape (11570, 64, 6)
y.shape (11570, 64, 3)


As can be seen, the `x` array has a shape of `(11570, 64, 6)`, representing `[no. samples, window length, no. signals]`. 

There are six signals in each sample (you can read more about the types of signals in the Data Exploration section, below). From index 0, to index 5, the signals are:

In [5]:
mill.signal_names

('AE_spindle', 'AE_table', 'vib_spindle', 'vib_table', 'smcDC', 'smcAC')

If you want to access the spindle vibration (`vib_spindle`) signal, you can index the numpy array like this:

In [10]:
# index into the 'vib_spindle' signal and show the shape
x[:,:,2].shape

(11570, 64)

The `y` array has a similar shape to `x`, but instead of signals, it contains the label and meta-data for each window. The shape of `y` is like this: `[no. samples, window length, no. label columns]`.

There are three label columns: y_labels_ids_times
* Label column 1 (index 0): The label for each sample (window). Either `0` for healthy, `1` for degraded, or `2` for worn (failed). More details provided in the Data Exploration section.
* Label column 2 (index 1): The ID of the sample. e.g. `1_0` for the first sample of the first window.
* Label column 3 (index 2): The timesteps for each data point in each sample.

To access the label for each sample, you can index the `y` array like this:

In [19]:
# use the colon in the first index to get all of the 11570 samples
# use 0 in the second index to only get the first label for each data point of each sample
# use 0 in the third index to get the label (either 0, 1, or 2)
y_label = y[:,0,0]
print("y_label.shape", y_label.shape)
y_label

y_label.shape (11570,)


array(['0', '0', '0', ..., '1', '1', '1'], dtype='<U32')

The data in the `y` array is stored with a dtype of `<u32`, a unicode string. You likely want to convert the label column to an integer, like this:

In [20]:
y_label.astype(int)

array([0, 0, 0, ..., 1, 1, 1])

In this example, each window (sample) has a length of 64. The signals are collected at 250 Hz. This means that the timesteps are spaced by 1/250 = 0.004 seconds. Here is what this looks like:

In [24]:
# show the timestep column, converted to `float` first
y[0,:,2].astype(float)

array([0.   , 0.004, 0.008, 0.012, 0.016, 0.02 , 0.024, 0.028, 0.032,
       0.036, 0.04 , 0.044, 0.048, 0.052, 0.056, 0.06 , 0.064, 0.068,
       0.072, 0.076, 0.08 , 0.084, 0.088, 0.092, 0.096, 0.1  , 0.104,
       0.108, 0.112, 0.116, 0.12 , 0.124, 0.128, 0.132, 0.136, 0.14 ,
       0.144, 0.148, 0.152, 0.156, 0.16 , 0.164, 0.168, 0.172, 0.176,
       0.18 , 0.184, 0.188, 0.192, 0.196, 0.2  , 0.204, 0.208, 0.212,
       0.216, 0.22 , 0.224, 0.228, 0.232, 0.236, 0.24 , 0.244, 0.248,
       0.252])

## Create Pandas Dataframe


In [1]:
import sys
# sys.path.append(r'/home/tim/Documents/PyPHM')
from pyphm.datasets.utils import _urlretrieve, extract_archive
from pyphm.datasets.ims import ImsDataLoad
from pathlib import Path
import hashlib
import py7zr
from rarfile import RarFile
import os


%load_ext autoreload
%autoreload 2

In [2]:
root_dir = Path.cwd().parent
print(root_dir)
path_data_raw_folder = Path(root_dir / 'data' )
print(path_data_raw_folder)
print(type(path_data_raw_folder))

/home/tim/Documents/PyPHM
/home/tim/Documents/PyPHM/data
<class 'pathlib.PosixPath'>


In [3]:
ims = ImsDataLoad(path_data_raw_folder, 'ims', download=False)

type(root) =  <class 'pathlib.PosixPath'>
/home/tim/Documents/PyPHM/data/ims


In [16]:
ims.download()

IMS.7z already exists.


In [4]:
ims.extract()

Extracting IMS.7z


In [8]:
l = list(Path(path_data_raw_folder / 'ims').glob('*.rar*'))

In [9]:
len(l)

0

In [3]:
extract_archive(Path.cwd() / '1st_test.rar')

'/home/tim/Documents/PyPHM/notebooks'

In [4]:
extract_archive(path_data_raw_folder / 'ims/IMS.7z')

'/home/tim/Documents/PyPHM/data/ims'

In [3]:
# extract .7z file
with py7zr.SevenZipFile(path_data_raw_folder / 'ims/IMS.7z', mode='r') as z:
    z.extractall()

In [6]:
# list all files in the cwd
print(os.listdir())


['test.jpg', '2nd_test.rar', 'Readme Document for IMS Bearing Data.pdf', 'test_ims.ipynb', 'test.ipynb', '3rd_test.rar', '1st_test.rar']


In [7]:
with RarFile(Path.cwd() / '1st_test.rar') as rf:
    rf.extractall()

In [ ]:
import tarfile

tar = tarfile.open(path_data_raw_folder / 'ims/IMS.7z')
tar.extractall()
tar.close()

In [8]:
extract_archive(path_data_raw_folder / 'ims/IMS.7z')

RuntimeError: Unknown compression or archive type: '.7z'.
Known suffixes are: '['.bz2', '.gz', '.tar', '.tbz', '.tbz2', '.tgz', '.xz', '.zip']'.

In [ ]:
ims = ImsDataLoad(path_data_raw_folder, download=True)

In [ ]:
mill = MillingPrepMethodA(path_data_raw_folder, download=False)

In [ ]:
x, y = mill.create_xy_arrays()
print("x.shape", x.shape)
print("y.shape", y.shape)

In [ ]:
y[0,0,:]

In [ ]:
df = mill.create_xy_dataframe()
df.head()

In [ ]:
df.shape

In [ ]:
y.shape

In [ ]:
x.shape

In [ ]:
# sys.path.append(root_dir / 'pyphm')
from pyphm.datasets.utils import _urlretrieve

In [ ]:
def file_as_bytes(file):
    with file:
        return file.read()

print(hashlib.md5(file_as_bytes(open(path_data_raw_folder / 'IMS.7z', 'rb'))).hexdigest())

In [ ]:
# _urlretrieve('https://files.realpython.com/media/Python-Imports_Watermarked.ae72c8a00197.jpg', 'test.jpg')

In [ ]:
import sys
sys.path

In [ ]:
import pyphm